In [6]:
import pandas as pd
import psycopg2
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('./BlinkIT Grocery Data.csv', 'blinkit_store'), # Added payments.csv for specific handling
]

# Folder containing the CSV files
folder_path = 'C:/Users/moham/Desktop/Data Analysis/2025 Data Analysis/SQL Zero - Hero/Blinkit'
# df = pd.read_csv(file_path, encoding='ISO-8859-1')


# Define a function to map pandas data types to SQL types
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INTEGER'
    elif pd.api.types.is_float_dtype(dtype):
        return 'REAL'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'
    else:
        return 'TEXT'

try:
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        host='localhost',
        user='postgres',
        password='mohammad',
        dbname='Blinkit Store',
        port='5432'  # typically 5432 for PostgreSQL
    )
    cursor = conn.cursor()
    print("Connected to the database.")

    # Loop over CSV files and load each into PostgreSQL
    for csv_file, table_name in csv_files:
        file_path = os.path.join(folder_path, csv_file)

        # Read the CSV file into a pandas DataFrame
        df = pd.read_csv('./BlinkIT Grocery Data.csv')

        # Replace NaN with None to handle SQL NULL
        df = df.where(pd.notnull(df), None)
        
        # Clean column names
        df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

        # Generate the CREATE TABLE statement with appropriate data types
        columns = ', '.join([f'"{col}" {get_sql_type(df[col].dtype)}' for col in df.columns])
        create_table_query = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({columns})'
        cursor.execute(create_table_query)
        print(f"Table `{table_name}` created or already exists.")

        # Insert DataFrame data into the PostgreSQL table
        for _, row in df.iterrows():
            # Convert row to tuple and handle NaN/None explicitly
            values = tuple(None if pd.isna(x) else x for x in row)
            sql = f"INSERT INTO \"{table_name}\" ({', '.join(['"' + col + '"' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(sql, values)

        # Commit the transaction for the current CSV file
        conn.commit()
        print(f"Data from `{csv_file}` inserted successfully into `{table_name}`.")

except psycopg2.Error as err:
    print(f"Error: {err}")
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")

Connected to the database.
Table `blinkit_store` created or already exists.
Data from `./BlinkIT Grocery Data.csv` inserted successfully into `blinkit_store`.
Database connection closed.


In [5]:
try:
    # Reconnect to the database if the connection is closed
    if conn.closed:
        conn = psycopg2.connect(
            host='localhost',
            user='postgres',
            password='mohammad',
            dbname='Blinkit Store',
            port=5432
        )
        cursor = conn.cursor()

    # Drop the table
    drop_table_query = 'DROP TABLE IF EXISTS "blinkit store"'
    cursor.execute(drop_table_query)
    conn.commit()
    print("Table 'blinkit store' has been deleted successfully.")
except psycopg2.Error as err:
    print(f"Error: {err}")
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")

Table 'blinkit store' has been deleted successfully.
Database connection closed.


In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv('./BlinkIT Grocery Data.csv')
df.head()

,Item Fat Content,Item Identifier,Item Type,Outlet Establishment Year,Outlet Identifier,Outlet Location Type,Outlet Size,Outlet Type,Item Visibility,Item Weight,Total Sales,Rating
0,Regular,FDX32,Fruits and Vegetables,2012,OUT049,Tier 1,Medium,Supermarket Type1,0.100014,15.10,145.4786,5.0
1,Low Fat,NCB42,Health and Hygiene,2022,OUT018,Tier 3,Medium,Supermarket Type2,0.008596,11.80,115.3492,5.0
2,Regular,FDR28,Frozen Foods,2010,OUT046,Tier 1,Small,Supermarket Type1,0.025896,13.85,165.0210,5.0
3,Regular,FDL50,Canned,2000,OUT013,Tier 3,High,Supermarket Type1,0.042278,12.15,126.5046,5.0
4,Low Fat,DRI25,Soft Drinks,2015,OUT045,Tier 2,Small,Supermarket Type1,0.033970,19.60,55.1614,5.0


In [9]:
for col in df.columns:
    print(col)

Item Fat Content
Item Identifier
Item Type
Outlet Establishment Year
Outlet Identifier
Outlet Location Type
Outlet Size
Outlet Type
Item Visibility
Item Weight
Total Sales
Rating


In [10]:
df.shape

(8523, 12)

In [19]:
df.isnull().sum()

Item Fat Content             0
Item Identifier              0
Item Type                    0
Outlet Establishment Year    0
Outlet Identifier            0
Outlet Location Type         0
Outlet Size                  0
Outlet Type                  0
Item Visibility              0
Total Sales                  0
Rating                       0
dtype: int64

In [ ]:
df.drop('Item Weight', axis=1, inplace=True)

KeyError: "['Item Weight'] not found in axis"

In [18]:
df.head()

,Item Fat Content,Item Identifier,Item Type,Outlet Establishment Year,Outlet Identifier,Outlet Location Type,Outlet Size,Outlet Type,Item Visibility,Total Sales,Rating
0,Regular,FDX32,Fruits and Vegetables,2012,OUT049,Tier 1,Medium,Supermarket Type1,0.100014,145.4786,5.0
1,Low Fat,NCB42,Health and Hygiene,2022,OUT018,Tier 3,Medium,Supermarket Type2,0.008596,115.3492,5.0
2,Regular,FDR28,Frozen Foods,2010,OUT046,Tier 1,Small,Supermarket Type1,0.025896,165.0210,5.0
3,Regular,FDL50,Canned,2000,OUT013,Tier 3,High,Supermarket Type1,0.042278,126.5046,5.0
4,Low Fat,DRI25,Soft Drinks,2015,OUT045,Tier 2,Small,Supermarket Type1,0.033970,55.1614,5.0


In [20]:
dd = pd.read_csv('./BlinkIT Grocery Data.csv')

In [21]:
dd.shape

(8523, 12)

In [22]:
dd.isnull().sum()

Item Fat Content                0
Item Identifier                 0
Item Type                       0
Outlet Establishment Year       0
Outlet Identifier               0
Outlet Location Type            0
Outlet Size                     0
Outlet Type                     0
Item Visibility                 0
Item Weight                  1463
Total Sales                     0
Rating                          0
dtype: int64

In [25]:
dd.drop('Item Weight', axis=1, inplace=True)
dd.to_csv('BlinkIT Grocery Data_backup.csv', index=False)

